In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

llm_api_key = os.getenv("ANTHROPIC_API_KEY")
api_max_tokens = 300


path_to_initPrompt_1 = r"Prompts\initial_prompt_1.txt"
path_to_pdf = "ReqView-Example_Software_Requirements_Specification_SRS_Document.pdf"
path_to_initPrompt_2 = r"Prompts\initial_prompt_2.txt"
path_to_reflectionPrompt = r"Prompts\reflection_prompt.txt"
path_to_finalPrompt = r"Prompts\final_prompt.txt"

path_to_output = "output_test_matrix.json"

In [2]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model="claude-3-7-sonnet-latest",
    temperature = 0.3,
    max_tokens = api_max_tokens,
    api_key = llm_api_key
)

In [12]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


In [ ]:
## content_prompt_initial
import base64

with open(path_to_pdf, "rb") as f:
    pdf_data = base64.standard_b64encode(f.read()).decode("utf-8")

with open(path_to_initPrompt_1,"r") as f:
    initPrompt_1 = f.read()

with open(path_to_initPrompt_2,"r") as f:
    initPrompt_2 = f.read()
    
content_prompt_initial = [
                {
                    "type": "text",
                    "text": initPrompt_1
                },
                {
                    "type": "document",
                    "source": {
                        "type": "base64",
                        "media_type": "application/pdf",
                        "data": pdf_data
                    },
                    "cache_control": {"type": "ephemeral"}
                },
                {
                    "type": "text",
                    "text": initPrompt_2
                }
            ]

In [14]:
## content_prompt_reflection
with open(path_to_reflectionPrompt,"r") as f:
    reflectionPrompt = f.read()

content_prompt_reflection = [
                {
                    "type": "text",
                    "text": reflectionPrompt
                }
            ]

In [15]:
## content_prompt_final
with open(path_to_finalPrompt,"r") as f:
    finalPrompt = f.read()
content_prompt_final = [
                {
                    "type": "text",
                    "text": finalPrompt
                }
            ]

In [16]:
from langchain_core.messages import AIMessage, HumanMessage

def write_initial_draft(state: AgentState):
    model_response = llm.invoke([HumanMessage(content=content_prompt_initial)])
    
    return {"messages": [HumanMessage(content=content_prompt_initial), model_response]}
    

In [17]:
def write_reflection(state: AgentState):
    prompt_messages = state['messages'] + [HumanMessage(content=content_prompt_reflection)] 
    model_response = llm.invoke(prompt_messages)

    return{"messages": [HumanMessage(content=content_prompt_reflection), model_response]}

In [18]:
def write_final(state: AgentState):
    prompt_messages = state['messages'] + [HumanMessage(content=content_prompt_final)] +  [AIMessage(content="{")]
    ## The AIMessage { is to tell Claude to only create JSON
    model_response = llm.invoke(prompt_messages)

    return{"messages":[HumanMessage(content=content_prompt_final), model_response]}

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(AgentState)
builder.add_node(write_initial_draft, "write_initial_draft")
builder.add_node(write_reflection, "write_reflection")
builder.add_node(write_final, "write_final")

builder.add_edge(START, "write_initial_draft")
builder.add_edge("write_initial_draft","write_reflection")
builder.add_edge("write_reflection","write_final")
builder.add_edge("write_final",END)

graph = builder.compile()

In [46]:
langchainConfig_messages = graph.invoke({"messages": []})

for message in langchainConfig_messages['messages']:
    print(message)

content=[{'type': 'text', 'text': 'You are tasked with generating comprehensive test matrices based on a given project document. This is a critical step in ensuring thorough testing coverage for the project. Your goal is to create exhaustive test cases that cover all aspects of the project requirements.\nHere is the project document you will be working with:'}, {'type': 'document', 'source': {'type': 'base64', 'media_type': 'application/pdf', 'data': '...'}, 'cache_control': {'type': 'ephemeral'}}, {'type': 'text', 'text': 'To complete this task, follow these steps:\n\n1. Carefully read and analyze the project document. Pay attention to:\n   - Functional requirements\n   - Non-functional requirements\n   - User scenarios\n   - Edge cases\n   - Potential error conditions\n\n2. Create a test matrix that covers all aspects of the project. For each test case, include:\n   - Test case ID\n   - Test case description\n   - Test steps\n   - Expected result\n   - Actual result (leave blank for 

In [61]:
import json

output_testCase_JSON_string = langchainConfig_messages['messages'][-1].content
print(output_testCase_JSON_string)


  "File Operations": [
    {
      "Test ID": "FO-001",
      "Test Case Description": "Create a new empty document",
      "Test Steps": "1. Launch application\n2. Click on 'Create New Document' option",
      "Expected Result": "A new empty document is created successfully",
      "Requirement ID": "DEMO-SRS-53",
      "Priority": "High",
      "Test Type": "Positive",
      "Preconditions": "Application is running"
    },
    {
      "Test ID": "FO-002",
      "Test Case Description": "Save changes before closing document",
      "Test Steps": "1. Make changes to document\n2. Attempt to close document",
      "Expected Result": "Application prompts user to save changes before closing",
      "Requirement ID": "DEMO-SRS-54",
      "Priority": "High",
      "Test Type": "Positive",
      "Preconditions": "Document is open and has unsaved changes"
    },
    {
      "Test ID": "FO-003",
      "Test Case Description": "Open document from file",
      "Test Steps": "1. Click 'Open File'

In [ ]:
output_testCase_JSON = json.loads(output_testCase_JSON_string)

def remove_newlines(obj):
    if isinstance(obj, dict):
        return {k: remove_newlines(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [remove_newlines(elem) for elem in obj]
    elif isinstance(obj, str):
        return obj.replace('\\n', ' ')
    else:
        return obj

output_testCase_JSON = remove_newlines(output_testCase_JSON)

with open(path_to_output, "w") as file:
    json.dump(output_testCase_JSON, file, indent=4)

JSONDecodeError: Extra data: line 2 column 20 (char 20)